In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [1]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash-preview-05-20", temperature=0)

generate_queries_step_back = prompt | llm | StrOutputParser()
question = "What is the name of the writter of the book and what the book says about ?"
generate_queries_step_back.invoke({"question": question})

'What information is available about this book?'

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

doc_path = "dev-data/Be_Good.pdf"
loader = PyPDFLoader(doc_path)

doc = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
doc_splits = text_splitter.split_documents(doc)

chromadb = Chroma.from_documents(
    documents=doc_splits,
    embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)
retriever = chromadb.as_retriever()

In [10]:
from langchain_core.runnables import RunnableLambda

# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        "step_back_context": generate_queries_step_back | retriever,
        "question": lambda x: x["question"]
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

In [11]:
chain.invoke({"question": question})

'The writer of the essay is **Paul Graham**.\n\nThe essay, titled "Be Good," discusses the concept of "being good" or "benevolence" not merely as a moral imperative, but as a powerful and effective strategy. Paul Graham argues that benevolence is a fundamental force that can lead to success, even in areas like business and technology. He suggests that:\n\n*   **Benevolence is a powerful force:** It\'s something one would be foolish to bet against, akin to a fundamental principle.\n*   **It provides a sense of mission:** Doing good for people can give a sense of purpose that makes individuals more resilient and dedicated.\n*   **It attracts help from others:** Being good makes other people want to assist you, which is described as an inborn human trait.\n*   **It\'s a practical strategy:** Graham emphasizes that he is not advocating for goodness in a sanctimonious way, but because "it works." He suggests it can serve as a guide for strategy and even a design specification for software, 